In [1]:
import json
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests

# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    profile.add_extension('/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/uBlock0@raymondhill.net.xpi')

    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    geko_path='/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/geckodriver'
    download_path='/Users/ruimaciel/Downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [4]:
# lets open booking:
dfolder='/Users/ruimaciel/Downloads'
geko_path='/Users/ruimaciel/Desktop/Barcelona/NLP_I/TA_sessions/2/geckodriver'
link='https://www.booking.com/index.en.html'

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

#eliminate the cookie bar
browser.find_element(by='xpath', value='//*[@id="onetrust-reject-all-handler"]').click()

In [3]:
place = "Barcelona"
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

from_day = '2024-05-25'
to_day = '2024-06-02'  # Change to your 'to_day'

calendar = 'button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'

browser.find_element('css selector', calendar).click()

path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
found_from_day = False
found_to_day = False
is_first_iteration = True
next_button_path_1 = '.f4552b6561'
next_button_path_2 = 'button.f38b6daa18:nth-child(2)'

while not (found_from_day and found_to_day):
    dates = browser.find_elements('xpath', path)
    for date in dates:
        if date.get_attribute("data-date") == from_day and not found_from_day:
            date.click()
            found_from_day = True
            break

        if date.get_attribute("data-date") == to_day and not found_to_day:
            date.click()
            found_to_day = True
            break

    if not (found_from_day and found_to_day):
        if is_first_iteration:
            browser.find_element('css selector', next_button_path_1).click()
            is_first_iteration = False
        else:
            browser.find_element('css selector', next_button_path_2).click()


search_button='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,search_button , type='xpath')
check_and_click(browser,search_button , type='xpath')


NameError: name 'browser' is not defined

In [28]:
def search_place_and_dates(browser, place, from_day, to_day):
    # Enter the place in the search field
    search1 = browser.find_element(by='xpath', value='//*[@id=":re:"]')
    search1.send_keys(place)

    # Click on the calendar
    calendar = 'button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
    browser.find_element('css selector', calendar).click()

    # Calendar navigation paths
    path = '//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
    next_button_path_1 = '.f4552b6561'
    next_button_path_2 = 'button.f38b6daa18:nth-child(2)'

    # Flags to track if dates are found
    found_from_day = False
    found_to_day = False
    is_first_iteration = True

    # Iterate to find from_day and to_day
    while not (found_from_day and found_to_day):
        dates = browser.find_elements('xpath', path)
        for date in dates:
            if date.get_attribute("data-date") == from_day and not found_from_day:
                date.click()
                found_from_day = True
                break

            if date.get_attribute("data-date") == to_day and not found_to_day:
                date.click()
                found_to_day = True
                break

        # Navigate the calendar
        if not (found_from_day and found_to_day):
            if is_first_iteration:
                browser.find_element('css selector', next_button_path_1).click()
                is_first_iteration = False
            else:
                browser.find_element('css selector', next_button_path_2).click()

    # Click the search button
    search_button = '/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'
    browser.find_element('xpath', search_button).click()

search_place_and_dates(browser, "Barcelona","2024-05-25", "2024-06-02")

KeyboardInterrupt: 

In [7]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        "/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[7]/button")
    return(int(a[-1].text))

pages = get_number_pages(browser)

print(pages)

40


In [34]:
pages=1

change_page_xpath = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
css = 'div.b16a89683f:nth-child(3) > button:nth-child(1) > span:nth-child(1) > span:nth-child(1)'
data = pd.DataFrame(columns=[])

# Go to first page
first_page_xpath='/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[2]/ol/li[1]/button'
check_and_click(browser,first_page_xpath , type='xpath')
for i in range(pages):
    # Save hotels info
    print(f'Page: {i + 1}')
    containers = browser.find_elements('xpath', '//div[@class="c066246e13"]')
    for hotel in containers:
        hotel_name = hotel.find_element('xpath', './/div[@class="f6431b446c a15b38c233"]').text
        try:
            hotel_rating = hotel.find_element('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]').text
        except:
            hotel_rating = np.nan
        try:
            promotion = hotel.find_element('xpath', './/div[@class="d17181842f"]').text
        except: 
            promotion = np.nan
        try:
            hotel_description_long= hotel.find_element('css selector', 'div.b1037148f8').text
        except:
            hotel_description_long = np.nan
        try:
            hotel_price = hotel.find_element('xpath', './/span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]').text
        except:
            hotel_price = np.nan
        try:
            url = hotel.find_element('xpath', './/a[@href]')
            hotel_url= url.get_attribute('href')
        except:
            hotel_url = np.nan
    
        new_row = {'Hotels': hotel_name, 'Ratings': hotel_rating, 'Price':hotel_price,'Promotion': promotion, 'Description': hotel_description_long,  'Link': hotel_url}
        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    # Change page with CSS Selector
    #browser.find_element('css selector', css).click()
    #time.sleep(2)
display(data)



Page: 1


,Hotels,Ratings,Price,Promotion,Description,Link
0,Casa Abamita,8.5,"€ 1,712",Only 1 left at this price on our site,Standard Interior Studio\nEntire studio • 1 ba...,https://www.booking.com/hotel/es/mayerling-aba...
1,INNSiDE by Meliá Barcelona Apolo,8.9,"€ 2,155",NaN,Innside Room\nBeds: 1 double or 2 singles\n8 n...,https://www.booking.com/hotel/es/innside-by-me...
2,SM Hotel Sant Antoni,8.6,"€ 1,524",Only 1 room left at this price on our site,Double or Twin Room\nBeds: 1 double or 2 singl...,https://www.booking.com/hotel/es/sant-antoni-b...
3,Hotel Lleó,8.2,"€ 1,849",Only 1 room left at this price on our site,Double or Twin Room with Balcony\nBeds: 1 doub...,https://www.booking.com/hotel/es/lleo.en-gb.ht...
4,Hotel del Mar,8.4,"€ 1,717",NaN,Double or Twin Room\n2 single beds\nFree cance...,https://www.booking.com/hotel/es/del-mar.en-gb...
5,Aparthotel Senator Barcelona,7.6,"€ 1,709",Only 2 rooms left at this price on our site,Double or Twin Room with Kitchen - Ground Floo...,https://www.booking.com/hotel/es/aparthotelcit...
6,Barcelona Fifteen central Aparthotel,7.8,"€ 1,352",Only 1 room left at this price on our site,Deluxe Double Room\n1 double bed\nOnly 1 room ...,https://www.booking.com/hotel/es/barcelona-fif...
7,Hotel Concordia Barcelona,8.2,"€ 2,179",Only 5 rooms left at this price on our site,Basic Double Room\n1 double bed\nFree cancella...,https://www.booking.com/hotel/es/concordia.en-...
8,Golden Hotel Barcelona,8.4,"€ 1,768",NaN,Double Room\nBeds: 1 double or 2 singles\nFree...,https://www.booking.com/hotel/es/golden-tulip-...
9,Hotel Lloret Ramblas,8.0,"€ 1,812",Only 3 rooms left at this price on our site,Superior Double Room\n1 double bed\nOnly 3 roo...,https://www.booking.com/hotel/es/lloret.en-gb....


In [36]:
pd.set_option('display.max_colwidth', None)
print(data["Description"])
pd.reset_option('display.max_colwidth')

0                                                    Standard Interior Studio\nEntire studio • 1 bathroom • 22m²\nBeds: 1 double or 2 singles\nFree cancellation\nOnly 1 left at this price on our site\n8 nights, 2 adults\n€ 3,121€ 1,712\nOriginal price € 3,121. Current price € 1,712.\nAdditional charges may apply\nSee availability
1                                                                                                                                                                                                     Innside Room\nBeds: 1 double or 2 singles\n8 nights, 2 adults\n€ 2,155\nPrice € 2,155\nAdditional charges may apply\nSee availability
2                                                                                                                                                  Double or Twin Room\nBeds: 1 double or 2 singles\nOnly 1 room left at this price on our site\n8 nights, 2 adults\n€ 1,524\nPrice € 1,524\nAdditional charges may apply\nSee availability
3   

Suggestion for controls, it is just a room (meaning that the others are apartments etc), is a promotion or not.